#### Append Sharpe, Sortino, Omega ratios to df_finviz_stocks_etfs

In [1]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")

date_str: 2025-04-25


In [2]:
SOURCE_PATH = f'..\data\{date_str}_df_finviz_stocks_etfs.parquet'
DES_PATH = f'..\data\{date_str}_df_finviz_n_ratios_stocks_etfs.parquet'
COMMON_TICKERS_PATH = f'..\data\{date_str}_df_common_tickers_stocks_etfs.parquet'

In [15]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 2500) 
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
# pd.set_option('display.width', None)        # Let the display adjust to the window
# pd.set_option('display.max_colwidth', None) # Show full content of each cell

In [4]:
# Load the parquet file into a DataFrame
df_finviz = pd.read_parquet(SOURCE_PATH)

# Display basic information about the DataFrame
print(f"df_finviz.shape: {df_finviz.shape}")
print("\nFirst few rows:")
print(df_finviz.head(20))

df_finviz.shape: (1560, 113)

First few rows:
        No.                                 Company               Index                  Sector                        Industry Country Exchange  Market Cap, M     P/E  Fwd P/E    PEG    P/S    P/B     P/C   P/FCF    Book/sh    Cash/sh  Dividend %  Dividend TTM Dividend Ex Date  Payout Ratio %       EPS  EPS next Q  EPS this Y %  EPS next Y %  EPS past 5Y %  EPS next 5Y %  Sales past 5Y %  Sales Q/Q %  EPS Q/Q %  EPS YoY TTM %  Sales YoY TTM %  Sales, M  Income, M  EPS Surprise %  Revenue Surprise %  Outstanding, M  Float, M  Float %  Insider Own %  Insider Trans %  Inst Own %  Inst Trans %  Short Float %  Short Ratio  Short Interest, M  ROA %   ROE %  ROI %  Curr R  Quick R  LTDebt/Eq  Debt/Eq  Gross M %  Oper M %  Profit M %  Perf Week %  Perf Month %  Perf Quart %  Perf Half %  Perf Year %  Perf YTD %  Beta       ATR  Volatility W %  Volatility M %  SMA20 %  SMA50 %  SMA200 %  50D High %  50D Low %  52W High %  52W Low %              52W

In [5]:
print(f'df_finviz.shape before removing duplicates: {df_finviz.shape}')
# Remove duplicates from df_finviz, keeping the first occurrence
df_finviz = df_finviz[~df_finviz.index.duplicated(keep='first')]

print(f'df_finviz.shape after removing duplicates: {df_finviz.shape}')

df_finviz.shape before removing duplicates: (1560, 113)
df_finviz.shape after removing duplicates: (1559, 113)


In [6]:
# Load the parquet file into a DataFrame
df_perf_ratios= pd.read_parquet(f'..\data\{date_str}_df_perf_ratios_stocks_etfs.parquet')

# Display basic information about the DataFrame
print(f"df_perf_ratios.shape: {df_perf_ratios.shape}")
print("\nFirst few rows:")
df_perf_ratios.head()

df_perf_ratios.shape: (1518, 24)

First few rows:


,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
A,5.153101,19.052913,2.697368,15.793771,65.559711,9.259747,3.204522,5.896718,1.619189,1.202716,1.867554,1.226958,-2.258456,-2.893618,0.672376,-3.871692,-4.635103,0.500579,-1.241176,-1.661680,0.804164,-0.846579,-1.118455,0.859515
AA,3.723058,11.141484,1.992562,10.880600,28.880362,4.638584,1.956660,2.954931,1.380855,1.063227,1.742029,1.200455,-2.404414,-3.160253,0.660904,-1.736037,-2.276252,0.738186,-1.412166,-1.848542,0.780054,-0.481224,-0.666162,0.922277
AAL,30.773939,3802.268850,339.733032,21.440824,15666.484213,1974.791483,0.627020,0.870148,1.105977,0.959973,1.686722,1.221849,-0.564022,-0.880908,0.893880,-2.831226,-3.910800,0.571980,-0.808785,-1.219591,0.848549,-0.487410,-0.718294,0.907541
AAON,36.487069,3802.268850,339.733032,47.280299,15666.484213,1974.791483,2.546433,3.924418,1.466622,4.647126,10.956734,2.263572,1.205968,2.089323,1.223243,-1.314793,-1.632743,0.775881,-0.564799,-0.742476,0.891811,0.141717,0.188445,1.029306
AAPL,17.945283,3802.268850,339.733032,25.731176,15666.484213,1974.791483,4.214191,6.719272,1.915683,2.638663,5.831754,1.629695,0.008446,0.013045,1.001637,-0.862319,-1.246745,0.843918,-0.305491,-0.438251,0.939919,0.708200,1.053839,1.149028


In [7]:
# Find items in df_finviz.index that are not in df_perf_ratios.index
missing_tickers = df_finviz.index[~df_finviz.index.isin(df_perf_ratios.index)]
print("Tickers in df_finviz but not in df_perf_ratios:")
print(missing_tickers)
print(f"\nTotal missing tickers: {len(missing_tickers)}")

Tickers in df_finviz but not in df_perf_ratios:
Index(['GEV', 'GOLD', 'SW', 'RDDT', 'VG', 'CRWV', 'VIK', 'RBRK', 'LINE', 'ULS', 'SOLV', 'TTAN', 'ALAB', 'SAIL', 'LTM', 'TEM', 'LOAR', 'SARO', 'BERY', 'SFD', 'BECN', 'TGTX', 'WAY', 'TLX', 'JGLO', 'TFX', 'GH', 'BMA', 'SRPT', 'DJT', 'NBIS', 'USM', 'COLD', 'FSK', 'CHRD', 'GKOS', 'LB', 'SMBS', 'BTC', 'IBDQ', 'MSTY'], dtype='object', name='Ticker')

Total missing tickers: 41


In [8]:
# Find common tickers (tickers present in both DataFrames' indices)
common_tickers = df_finviz.index[df_finviz.index.isin(df_perf_ratios.index)].tolist() #Convert pandas index to list

print("\nTickers common to both df_finviz and df_perf_ratios:")
print(common_tickers)
print(f"\nTotal common tickers: {len(common_tickers)}")

# Create the empty DataFrame with the common tickers as the index
df_common_tickers = pd.DataFrame(index=common_tickers)
df_common_tickers.to_parquet(COMMON_TICKERS_PATH, engine='pyarrow', compression='zstd')
print(f'Saved df_common_tickers to {COMMON_TICKERS_PATH}')


Tickers common to both df_finviz and df_perf_ratios:
['AAPL', 'MSFT', 'NVDA', 'AMZN', 'GOOGL', 'GOOG', 'META', 'BRK-A', 'BRK-B', 'TSLA', 'AVGO', 'TSM', 'LLY', 'WMT', 'JPM', 'V', 'VOO', 'SPY', 'IVV', 'MA', 'XOM', 'NFLX', 'VTI', 'COST', 'ORCL', 'UNH', 'PG', 'JNJ', 'HD', 'ABBV', 'SAP', 'KO', 'BAC', 'QQQ', 'BABA', 'ASML', 'PM', 'PLTR', 'TMUS', 'CRM', 'TM', 'CVX', 'WFC', 'MCD', 'CSCO', 'ABT', 'IBM', 'AZN', 'NVS', 'LIN', 'GE', 'NVO', 'MRK', 'HSBC', 'SHEL', 'NOW', 'T', 'MS', 'AXP', 'ISRG', 'HDB', 'ACN', 'PEP', 'VZ', 'INTU', 'GS', 'RTX', 'RY', 'QCOM', 'DIS', 'UBER', 'BX', 'TMO', 'BKNG', 'AMD', 'ADBE', 'UL', 'PGR', 'AMGN', 'BSX', 'SPGI', 'SONY', 'SCHW', 'VEA', 'TXN', 'VUG', 'CAT', 'PDD', 'MUFG', 'TJX', 'DHR', 'BLK', 'SYK', 'NEE', 'BA', 'IEFA', 'PFE', 'GILD', 'VTV', 'CMCSA', 'HON', 'SNY', 'C', 'UNP', 'VRTX', 'TTE', 'SHOP', 'BND', 'DE', 'SPOT', 'LOW', 'AGG', 'AMAT', 'BHP', 'ARM', 'ADP', 'PANW', 'IBN', 'COP', 'BUD', 'ETN', 'MELI', 'LMT', 'CB', 'SAN', 'TD', 'MMC', 'MDT', 'CRWD', 'KKR', 'ENB', 'GLD

In [9]:
print(f'df_finviz.shape before removing missing tickers: {df_finviz.shape}')
# Remove rows where the index matches missing_tickers
df_finviz = df_finviz.drop(missing_tickers)
print(f'df_finviz.shape after removing missing tickers: {df_finviz.shape}')


df_finviz.shape before removing missing tickers: (1559, 113)
df_finviz.shape after removing missing tickers: (1518, 113)


In [10]:
print(f'df_perf_ratios.shape before removing duplicate tickers: {df_perf_ratios.shape}')
# Remove duplicates from df_perf_ratios, keeping the first occurrence
df_perf_ratios = df_perf_ratios[~df_perf_ratios.index.duplicated(keep='first')]
print(f'df_perf_ratios.shape after removing duplicate tickers: {df_perf_ratios.shape}')


df_perf_ratios.shape before removing duplicate tickers: (1518, 24)
df_perf_ratios.shape after removing duplicate tickers: (1518, 24)


In [20]:
# Find duplicate columns *before* adding suffixes
duplicate_cols = list(set(df_finviz.columns) & set(df_perf_ratios.columns))  # Intersection

if duplicate_cols:
    print(f"Error: Duplicate columns found: {duplicate_cols}")
    raise ValueError(f"Duplicate columns found: {duplicate_cols}")

# Now, perform the join (WITHOUT suffixes)
try:
    df = df_finviz.join(df_perf_ratios)  # Simplified join
    print("Join successful!")
    print(f'df.head():\n{df.head()}') # Print first few lines.
    print('\n')
    print(f'df.info():\n{df.info()}') # Print info about the DataFrame.
except ValueError as e:
    print(f"Join failed: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Join successful!
df.head():
        No.                Company               Index                  Sector                        Industry Country Exchange  Market Cap, M    P/E  Fwd P/E   PEG    P/S    P/B    P/C  P/FCF  Book/sh  Cash/sh  Dividend %  Dividend TTM Dividend Ex Date  Payout Ratio %    EPS  EPS next Q  EPS this Y %  EPS next Y %  EPS past 5Y %  EPS next 5Y %  Sales past 5Y %  Sales Q/Q %  EPS Q/Q %  EPS YoY TTM %  Sales YoY TTM %  Sales, M  Income, M  EPS Surprise %  Revenue Surprise %  Outstanding, M  Float, M  Float %  Insider Own %  Insider Trans %  Inst Own %  Inst Trans %  Short Float %  Short Ratio  Short Interest, M  ROA %   ROE %  ROI %  Curr R  Quick R  LTDebt/Eq  Debt/Eq  Gross M %  Oper M %  Profit M %  Perf Week %  Perf Month %  Perf Quart %  Perf Half %  Perf Year %  Perf YTD %  Beta    ATR  Volatility W %  Volatility M %  SMA20 %  SMA50 %  SMA200 %  50D High %  50D Low %  52W High %  52W Low %        52W Range  All-Time High %  All-Time Low %    RSI  Earning

In [12]:
df.to_parquet(DES_PATH, engine='pyarrow', compression='zstd')

print(f"Merged dataframe saved to {DES_PATH}")

Merged dataframe saved to ..\data\2025-04-25_df_finviz_n_ratios_stocks_etfs.parquet
